In [2]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dense
from sklearn.preprocessing import StandardScaler

In [1]:
from google.colab import files
uploaded = files.upload()

Saving 20230727_195816_hn_sharings.json to 20230727_195816_hn_sharings.json


In [3]:
with open('20230727_195816_hn_sharings.json', 'r') as file:
    data = json.load(file)

In [4]:
conversations = []
labels = []

for source in data['Sources']:
    for chat in source['ChatgptSharing']:
        if 'Conversations' not in chat:
            continue
        for conv in chat['Conversations']:
          conversations.append(conv['Prompt'] + ' ' + conv['Answer'])
          labels.append(len(conv['Prompt'].split()) + len(conv['Answer'].split()))

In [5]:
df = pd.DataFrame({'Conversations': conversations, 'Labels': labels})

In [6]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [7]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(train_df['Conversations'])
X_test_tfidf = tfidf_vectorizer.transform(test_df['Conversations'])

In [8]:
lr_model = LinearRegression()
lr_model.fit(X_train_tfidf, train_df['Labels'])

LinearRegression()

In [9]:
predictions_lr = lr_model.predict(X_test_tfidf)
mse_lr = mean_squared_error(test_df['Labels'], predictions_lr)
print(f'Mean Squared Error (Linear Regression): {mse_lr}')

Mean Squared Error (Linear Regression): 19420.579936559156


In [10]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train_tfidf.shape[1]))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error')

In [11]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_tfidf.toarray())
X_test_scaled = scaler.transform(X_test_tfidf.toarray())

In [12]:
model.fit(X_train_scaled, train_df['Labels'], epochs=100
          , batch_size=32, validation_split=0.2)

Epoch 1/100
13/13 [==============================] - 3s 52ms/step - loss: 123992.6953 - val_loss: 240416.8750
Epoch 2/100
13/13 [==============================] - 0s 23ms/step - loss: 119368.5312 - val_loss: 241014.7188
Epoch 3/100
13/13 [==============================] - 0s 23ms/step - loss: 115923.5703 - val_loss: 241652.8281
Epoch 4/100
13/13 [==============================] - 0s 29ms/step - loss: 112582.4219 - val_loss: 241851.5156
Epoch 5/100
13/13 [==============================] - 0s 20ms/step - loss: 108778.5703 - val_loss: 241560.1875
Epoch 6/100
13/13 [==============================] - 0s 22ms/step - loss: 104825.3438 - val_loss: 240712.2969
Epoch 7/100
13/13 [==============================] - 0s 14ms/step - loss: 100666.3047 - val_loss: 239351.4219
Epoch 8/100
13/13 [==============================] - 0s 19ms/step - loss: 96255.7812 - val_loss: 237797.0312
Epoch 9/100
13/13 [==============================] - 0s 12ms/step - loss: 91965.0859 - val_loss: 235979.1094
Epoch 10/100

In [13]:
predictions_nn = model.predict(X_test_scaled)
mse_nn = mean_squared_error(test_df['Labels'], predictions_nn)
print(f'Mean Squared Error (Neural Network): {mse_nn}')

4/4 [==============================] - 0s 3ms/step
Mean Squared Error (Neural Network): 48878.55811849744


In [16]:
predicted_duration_nn = model.predict(X_test_scaled)
print(f'Predicted Duration: {predicted_duration_nn.mean()}')

4/4 [==============================] - 0s 6ms/step
Predicted Duration: 202.61773681640625


In [17]:
predicted_duration_lr = lr_model.predict(X_test_tfidf)
print(f'Predicted Duration (Linear Regression): {predicted_duration_lr.mean()}')

Predicted Duration (Linear Regression): 239.8335868057432
